## ARC 380 / CEE 380 / ROB 380 – Introduction to Robotics for Digital Fabrication  
### Session 8 Workshop: Robot Description, Visualization, and Simulation in ROS  
Princeton University, Spring 2026  
Professor: Arash Adel | TA: Daniel Ruan

---

# Overview

By the end of this workshop you should be able to:

1. Explain what a **ROS 2 package** is and why we use **colcon** workspaces.
2. Read a robot description package (URDF + meshes) and launch a basic visualization.
3. Use **RViz2** to visualize a robot and **joint_state_publisher_gui** to jog joints.
4. Implement a small **rclpy** node that publishes `sensor_msgs/JointState` using **runtime parameters**.


Clone the course repository from GitHub:

```bash
git clone https://github.com/AdelResearchGroup/arc380_s26.git
```

# 1. ROS 2 packages and colcon

## 1.1 What is a ROS 2 package?

A **package** is the smallest unit of build + distribution in ROS. It typically contains:
- source code (`.py` or `.cpp`)
- metadata (`package.xml`)
- build / install rules (`setup.py` for Python, `CMakeLists.txt` for C++)

Other contents that we will typical encounter:
- launch files (`launch/`)
- source code (`src/`)
- configuration files (`config/`)
- robot description files (`urdf/`, `meshes/`)

Why packages matter:
- consistent dependency tracking
- reproducible builds
- discoverable executables (`ros2 run <pkg> <exe>`, `ros2 launch <pkg> <launch_file>`)
- sharing + reusing code and robot models


## 1.2 Colcon workspace anatomy

A **colcon workspace** is usually:

```text
~/arc380_s26/
  src/                 # packages live here
    some_pkg/
    another_pkg/
  build/               # generated (do not edit)
  install/             # generated (what you "source")
  log/                 # generated
```

Key idea:
- You **build** the workspace once (or whenever you change code).
- You **source** the install setup file each new terminal.

```bash
# from the workspace root:
colcon build
call install/setup.bat      # cmd
source install/setup.bash   # bash
source install/setup.zsh    # zsh
```

If you forget to source, ROS won’t “see” your packages.


## 1.3 Creating a minimal Python package

From your workspace root:

```bash
cd src      # Change Directory (cd) to the src folder

# Create a Python package with ament_python build type
ros2 pkg create --build-type ament_python arc380_session08 --dependencies rclpy
```

What this command generates (typical):
- `package.xml` (dependencies)
- `setup.py`, `setup.cfg`
- `arc380_session08/` Python module directory
- `resource/arc380_session08` (ament index marker)

Build + source:

```bash
cd ..           # cd up one directory (back to the workspace root)
colcon build

call install/setup.bat      # cmd
source install/setup.bash   # bash
source install/setup.zsh    # zsh
```


## 1.4 Running executables

For Python packages, `ros2 run` launches Python entry points defined in `setup.py`.

Example entry point pattern:

```python
entry_points={
    "console_scripts": [
        "my_node = arc380_session08.my_node:main",
    ],
}
```

That means:

```bash
ros2 run arc380_session08 my_node
```

runs the `main()` function inside `arc380_session08/my_node.py`.


---

# 2. Robot description packages

A **robot description package** typically contains:
- `urdf/` (URDF or Xacro)
- `meshes/visual/` (what you see)
- `meshes/collision/` (simplified geometry for collision checking)
- `launch/` (RViz + robot_state_publisher launches)
- sometimes `config/` (RViz configs)

## 2.1 URDF
**URDF** (Unified Robot Description Format) is an XML-based format used in ROS to describe a robot’s kinematic structure and physical properties. It defines:
- The robot’s rigid bodies (links)
- The connections between them (joints)
- The geometric representation (meshes or primitives)
- Optional inertial properties (for physics simulation)

URDF **does not**:
- Perform forward kinematics
- Simulate physics
- Plan motion

Instead, it defines the static model that other ROS nodes use.

In ROS visualization, the URDF is usually published on the `robot_description` parameter by `robot_state_publisher`.


## 2.2 Xacro

In practice, many robot description files are not written as raw `.urdf` files. Instead, they are written in Xacro.

Xacro (XML Macro Language) is a preprocessor for URDF. It allows you to:
- Define reusable macros
- Use parameters
- Define variables
- Avoid repetitive XML
- Create configurable robot variants

A `.xacro` file is converted into a standard `.urdf` file before being used.

For example:
```bash
xacro irb120_3_60_macro.xacro > irb120.urdf
```

The resulting expanded URDF is what `robot_state_publisher` actually consumes.

## 2.3 The `<robot>` root element

Every URDF/Xacro starts with:

```xml
<robot name="irb120">
  ...
</robot>
```

Everything (links, joints, materials, macros) lives inside this root element.

## 2.4 Links

A `<link>` represents a rigid body in the robot.

**Basic Structure**

```xml
<link name="link_1">
  <visual>...</visual>
  <collision>...</collision>
  <inertial>...</inertial>
</link>
```

Each link can define three different “aspects”:
- Visual element
- Collision element
- Inertial element

### 2.4.1 Visual element

```xml
<visual>
  <origin xyz="0 0 0" rpy="0 0 0"/>
  <geometry>
    <mesh filename="package://abb_irb120_description/meshes/visual/link_1.stl"/>
  </geometry>
</visual>
```

Key components:
- `<origin xyz="" rpy="">`
  - Transform from the link frame to the mesh frame
  - `xyz` → translation (meters)
  - `rpy` → roll, pitch, yaw (radians)
- `<geometry>`
  - Can be:
    - `<box>`
    - `<cylinder>`
    - `<sphere>`
    - `<mesh>`
- `<mesh filename="package://...">`
  - `package://` resolves using ROS package indexing
  - Mesh files are usually:
    - `.stl`
    - `.obj`
    - `.dae`
  - Visual meshes are often high-resolution


### 2.4.2 Collision element

```xml
<collision>
  <origin xyz="0 0 0" rpy="0 0 0"/>
  <geometry>
    <mesh filename="package://abb_irb120_description/meshes/collision/link_1.stl"/>
  </geometry>
</collision>
```

Collision geometry is typically:
- Simplified
- Lower-resolution
- Faster for collision checking

**Important**:
- Visual ≠ Collision.
- Collision meshes are optimized for computation.


### 2.4.3 Inertial Element

```xml
<inertial>
  <origin xyz="0 0 0" rpy="0 0 0"/>
  <mass value="3.2"/>
  <inertia ixx="..." ixy="..." ixz="..."
           iyy="..." iyz="..."
           izz="..."/>
</inertial>
```

This defines:
- Mass (kg)
- 3×3 inertia tensor

RViz does not use this, but physics engines (Gazebo, Isaac, etc.) do for simulation.

## 2.5 Joints

A `<joint>` defines how two links connect.

```xml
<joint name="joint_1" type="revolute">
  <parent link="base_link"/>
  <child link="link_1"/>
  <origin xyz="0 0 0.29" rpy="0 0 0"/>
  <axis xyz="0 0 1"/>
  <limit lower="-3.14" upper="3.14"
         effort="100" velocity="1.5"/>
</joint>
```


### 2.5.1 Joint types

Common types:
- `fixed`
- `revolute` (bounded rotation)
- `continuous` (unbounded rotation)
- `prismatic` (linear motion)
- `floating` (6-DOF)
- `planar`

For industrial arms like the ABB IRB 120, all joints are typically **revolute**.


### 2.5.2 Parent / Child
URDF forms a **tree**, not a graph.

Each joint defines:
- A parent link
- A child link

This builds the full kinematic chain:

```txt
base_link → joint_1 → link_1
link_1    → joint_2 → link_2
...
```


### 2.5.3 Origin
```xml
<origin xyz="x y z" rpy="r p y"/>
```

This defines the transform from the parent link frame to the joint frame. This is critical for defining the robot's kinematic geometry and computing forward kinematics (more on this next week).


### 2.5.4 Axis
```xml
<axis xyz="0 0 1"/>
```

Defines the axis of motion in the joint frame.
- For revolute joints, this is the rotation axis.
- For prismatic joints, this is the translation direction.

### 2.5.5 Limits
```xml
<limit lower="..." upper="..."
       effort="..." velocity="..."/>
```

Defines:
- Joint bounds (radians or meters)
- Max effort (Nm or N)
- Max velocity

These values connect directly to:
- Configuration space constraints
- Motion planning constraints

## 2.6 Frames in URDF
In addition to physical links with geometry, URDF often defines pure reference frames. These are implemented as links with no visual or collision geometry, connected by fixed joints.

These frames are critical for:
- Kinematics
- Tool definition
- Calibration
- Motion planning
- Integration with external systems

Example reference frame:

```xml
<link name="base"/>
```

No `<visual>`, `<collision>`, or `<inertial>` elements are required.

It is then attached using a fixed joint:

```xml
<joint name="base_link-base" type="fixed">
  <origin xyz="0 0 0" rpy="0 0 0"/>
  <parent link="base_link"/>
  <child link="base"/>
</joint>
```

This creates a rigid transform between `base_link` and `base`.

---

# 3. RViz2

RViz2 is ROS 2’s primary 3D visualization tool. It allows you to visualize:
- Robot models (URDF)
- Coordinate frames (TF)
- Sensor data (point clouds, laser scans, images)
- Markers and debugging visuals
- Planning results

RViz does **not** compute kinematics or physics. It is purely a visualization tool that renders data being published in the ROS graph.

For robot models specifically:
- URDF defines the structure and geometry.
- `/joint_states` defines the configuration.
- `/tf` defines the transforms between frames.
- RViz renders the robot in 3D using this information.

You can think of RViz as a live window into the current state of the ROS system.

## 3.1 Setup
To visualize a robot, RViz2 typically needs:
- TF transforms (published by `robot_state_publisher`)
- joint states (`/joint_states`) (e.g., from `joint_state_publisher_gui`)
- the URDF string on the `robot_description` parameter

Common minimal setup:
- `robot_state_publisher` (reads URDF, publishes TF)
- `joint_state_publisher_gui` (publishes `/joint_states`)
- `rviz2` (visualizes)

## 3.2 Launch the robot visualization
Run the provided launch file:

```bash
ros2 launch abb_irb120_description display.launch.py
```

If RViz opens but you don’t see the robot, check:
1. **Fixed Frame** in RViz (often `base_link` or `world`)
2. Add a **RobotModel** display and ensure it uses `robot_description`
3. Ensure `/tf` and `/joint_states` are being published:

```bash
ros2 topic list
ros2 topic echo /joint_states --once
ros2 topic echo /tf --once
```

## 3.3 Jogging joints with joint_state_publisher_gui
In a separate terminal:

```bash
ros2 run joint_state_publisher_gui joint_state_publisher_gui
```

Move sliders to change joint values and watch the robot in RViz.

Important:
- Only run **one** thing publishing `/joint_states` at a time.


# 3.4 Publishing joint states with rclpy

Rather than using the `joint_state_publisher_gui` to manually publish joint angles, we can also do the same thing programmatically with a Python node.

An example script is provided in `workshops/Session08/joint_state_ramp.py`, which moves a joint on the robot back and forth between fixed limits and at a fixed speed.

**Quick exercise**: Copy the script into our new `arc380_session08` package and create a new entry point that runs the script.

---

# 4. ROS Parameters

A **parameter** is a named value stored on a node  used to configure behavior.

Why parameters are useful:
- Change node behavior without editing code
- Tune values while the node is running (great for demos + debugging)
- Configure nodes from launch files / YAML
- Make scripts reusable across different robots (different joint names, limits, rates, etc.)

Two ways to change parameters:
- `ros2 param set ...`
- `rqt` → Plugins → Configuration → Dynamic Reconfigure

**Code along**: Convert `speed` in the `JointStateRamp` node to a parameter.

**Exercise**: Convert the other hard-coded values into configurable parameters:
- `joint_name`
- `min_value`
- `max_value`

---

# 5. Troubleshooting checklist

## “Package not found” / “executable not found”
- Did you run `colcon build` from the workspace root?
- Did you source the workspace in this terminal?

```bash
call install/setup.bat      # cmd
source install/setup.bash   # bash
source install/setup.zsh    # zsh
```

## Robot doesn’t move in RViz
- Ensure only ONE publisher on `/joint_states`:
  - stop `joint_state_publisher_gui` if your script is running (or vice versa)
- Check that messages exist:

```bash
ros2 topic echo /joint_states --once
```

## RViz shows nothing
- Check RViz **Fixed Frame**
- Ensure `robot_state_publisher` is running and publishing TF:

```bash
ros2 topic echo /tf --once
```


# 6. References + Additional Resources

- ROS 2 command line tools: `ros2 -h`, `ros2 topic -h`, `ros2 pkg -h`
- ROS 2 URDF Documentation + Tutorials: https://docs.ros.org/en/jazzy/Tutorials/Intermediate/URDF/URDF-Main.html
- ROS 2 RViz2 Documentation: https://docs.ros.org/en/jazzy/Tutorials/Intermediate/RViz/RViz-Main.html


# 7. Assignments

- Quiz 5: Robot Terminology + Configuration Space (due Sun, Feb 22, 11:59 pm)
- Assignment 3: Configuration-Space Waypoint Player (due Tues, Feb 24, 11:59 pm)
- No new quiz for this workshop